In [1]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [2]:
attributes = {}

In [3]:
## Sanity check calculation on the total number of possible combos
def num_possible_combinations(all_attrs):
    total_combos = 1
    for category_name in all_attrs:
        category = all_attrs[category_name]
        num_files_in_category = 0
        for group_name in category:
            group = category[group_name]
            num_files_in_category += len(group["files"])
        total_combos *= num_files_in_category
    return total_combos

print(num_possible_combinations(attributes))

625


In [14]:
# Number of random unique images we want to generate
TOTAL_IMAGES = 25

In [15]:
## Generate Traits

all_images = [] 

# A recursive function to generate unique image combinations.
# Keep track of trait counts as we generate.
def create_new_image():    
    new_image = {"attributes": []}
    
    # For each trait category, select a random trait based on the weightings 
    for category_name in attributes:
        category = attributes[category_name]
        
        # need to pick between groups if there are multiple
        num_files_in_category = 0
        group_name_to_use = ""
        group_names = []
        num_files_per_group = []
        weights_per_group = []
        for group_name in category:
            group_names.append(group_name)
            group = category[group_name]
            group_files = group["files"]
            num_files_in_category += len(group_files)
            num_files_per_group.append(len(group_files))
        # calculate implicit group weights via num_files_in_group/num_files_in_category but use whole nums
        for i in range(len(num_files_per_group)):
            g = num_files_per_group[i]
            c = num_files_in_category
            weights_per_group.append(g/c*100)
            
        group_name_to_use = random.choices(group_names, weights_per_group)[0]
        category[group_name_to_use]
        group = category[group_name_to_use]
        
        # pick the item to use for this attribute category
        filename = random.choices(group["files"], group["weights"])[0]
        new_image["attributes"].append({"trait_type": category_name, "group": group_name_to_use, "value": filename})
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES):
    all_images.append(create_new_image())


print("total images generated: ", TOTAL_IMAGES)

total images generated:  25


In [16]:
# Returns true if all images are unique.
# This is just a sanity check.. the create_new_image function should be generating unique images already.
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [17]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

print("Final tokenId: ", i-1)

Final tokenId:  24


In [18]:
for img in all_images:
    print(str(img) + "\n")

{'attributes': [{'trait_type': 'accessory', 'group': '4-accessory', 'value': 'red bow'}, {'trait_type': 'background', 'group': '1-background', 'value': 'pink'}, {'trait_type': 'body', 'group': '2-body', 'value': 'plain'}, {'trait_type': 'expression', 'group': '3-expression', 'value': 'excited'}], 'tokenId': 0}

{'attributes': [{'trait_type': 'accessory', 'group': '4-accessory', 'value': 'gold chain'}, {'trait_type': 'background', 'group': '1-background', 'value': 'pink'}, {'trait_type': 'body', 'group': '2-body', 'value': 'red'}, {'trait_type': 'expression', 'group': '3-expression', 'value': 'derp'}], 'tokenId': 1}

{'attributes': [{'trait_type': 'accessory', 'group': '4-accessory', 'value': 'pizza'}, {'trait_type': 'background', 'group': '1-background', 'value': 'red dots'}, {'trait_type': 'body', 'group': '2-body', 'value': 'green'}, {'trait_type': 'expression', 'group': '3-expression', 'value': 'frustrated'}], 'tokenId': 2}

{'attributes': [{'trait_type': 'accessory', 'group': '4-ac

In [19]:
from collections import Counter

counters = {}
for category_name in attributes:
    counters[category_name] = Counter()

for image in all_images:
    for trait in image["attributes"]:
        category_name = trait["trait_type"]
        filename = trait["value"]
        counters[category_name][filename] += 1

print("total number of images: ", TOTAL_IMAGES)
print("")

for category_name in counters:
    counter = counters[category_name]
    print(category_name,"weights ...")
    num_items_in_category = len(counter)
    
    category = attributes[category_name]
    
    all_filenames_in_category = []
    
    expected_weights_by_filename = {}

    num_files_in_category = 0
    num_files_by_group = {}
    
    expected_num_items_in_category = 0
    for group_name in category:
        group = category[group_name]
        all_filenames_in_category += group["files"]
        num_files = len(group["files"])
        num_files_in_category += num_files
        num_files_by_group[group_name] = num_files
        
        expected_num_items_in_category += len(attributes[category_name][group_name]["files"])
        
    print("num items in category : actual - {} expected - {}".format(num_items_in_category, expected_num_items_in_category))
          
    # calculate implicit group weights via num_files_in_group/num_files_in_category but use whole nums
    for group_name in category:
        g = num_files_by_group[group_name]
        c = num_files_in_category
        group_weight = g/c
        
        group = category[group_name]
        for i, filename in enumerate(group["files"]):
            file_weight = group["weights"][i]
            expected_weights_by_filename[filename] = round(float(group_weight * file_weight),2)
    
    for filename in all_filenames_in_category:
        actual_weight = counter[filename] / TOTAL_IMAGES
        expected_weight = expected_weights_by_filename[filename]
        print(filename, ":", "actual -", round(float(actual_weight*100),2),"expected -",expected_weight)
    print("")

total number of images:  25

accessory weights ...
num items in category : actual - 5 expected - 5
gold chain : actual - 36.0 expected - 20.0
green beanie : actual - 12.0 expected - 20.0
iridium dagger : actual - 16.0 expected - 20.0
pizza : actual - 20.0 expected - 20.0
red bow : actual - 16.0 expected - 20.0

background weights ...
num items in category : actual - 5 expected - 5
blue : actual - 8.0 expected - 20.0
navy : actual - 20.0 expected - 20.0
pink : actual - 20.0 expected - 20.0
purple : actual - 24.0 expected - 20.0
red dots : actual - 28.0 expected - 20.0

body weights ...
num items in category : actual - 5 expected - 5
blue : actual - 12.0 expected - 20.0
green : actual - 32.0 expected - 20.0
plain : actual - 16.0 expected - 20.0
rainbow neon : actual - 28.0 expected - 20.0
red : actual - 12.0 expected - 20.0

expression weights ...
num items in category : actual - 5 expected - 5
angry : actual - 4.0 expected - 20.0
derp : actual - 20.0 expected - 20.0
excited : actual - 4

In [23]:
def sortkey(group_name):
    return int(group_name.split("-")[0])

#### Generate Images    
for image in all_images:
    layers = []
    
    sorted_traits = sorted(image["attributes"], key=lambda x: sortkey(x["group"]))
    for trait in sorted_traits:
        filepath = f'./attributes/{trait["trait_type"]}/{trait["group"]}/{trait["value"]}.png'
        opened = Image.open(filepath).resize((512, 512)).convert('RGBA')
        layers.append(opened)
    
    composite = Image.alpha_composite(layers[0], layers[1])
    for i in range(2, len(layers)):
        composite = Image.alpha_composite(composite, layers[i])
        
    rgb_im = composite.convert('RGB')
    filename = str(image["tokenId"]) + ".png"
    rgb_im.save("./images/" + filename)
    print("generated: ", filename)
    

generated:  0.png
generated:  1.png
generated:  2.png
generated:  3.png
generated:  4.png
generated:  5.png
generated:  6.png
generated:  7.png
generated:  8.png
generated:  9.png
generated:  10.png
generated:  11.png
generated:  12.png
generated:  13.png
generated:  14.png
generated:  15.png
generated:  16.png
generated:  17.png
generated:  18.png
generated:  19.png
generated:  20.png
generated:  21.png
generated:  22.png
generated:  23.png
generated:  24.png


In [24]:
#### Grab all generated filenames from the images folder. Sort by image number.
from os import walk

filenames = next(walk("./images"), (None, None, []))[2]
filenames = list(filter(lambda x: x[0] != '.', filenames))
print(len(filenames))
filenames = sorted(filenames, key=lambda x: int(x[:len(x)-4]))
print(filenames)

25
['0.png', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png']


In [29]:
#### Generate Incomplete Metadata for each Image

metadata = []

for image in all_images:
    token_id = image["tokenId"]
    image["description"] = ""
    image["name"] = f'#{token_id}'
    image["image"] = f'ipfs://QmPJRxuauHGeKxvZ2mUTUtjezd27F55CwCmPqWYyk454P2/{token_id}.png'
    image["external_url"] = f'https://ipfs.io/ipfs/QmPJRxuauHGeKxvZ2mUTUtjezd27F55CwCmPqWYyk454P2/{token_id}.png'
    with open(f'./metadata/{str(token_id)}', 'w') as outfile:
        json.dump(image, outfile, indent=4)
    
    metadata.append(image)
    
    
#### Uncomment if you want to generate a top-level file with ALL metadata objects in a list

# METADATA_FILE_NAME = './metadata/all-traits.json'; 
# with open(METADATA_FILE_NAME, 'w') as outfile:
#    json.dump(metadata, outfile, indent=4)
